In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [33]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkDBwls as spark
from mining.modules.assets.etp import logReturns as log

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    log = importlib.reload(log)
    spark = importlib.reload(spark)
    
__desc__ = "analyze crypto market capitalization time series data"
clsSpark = spark.SQLWorkLoads(desc=__desc__)
clsROR = log.RatioOfReturns(desc=__desc__)
''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional LOGRETURNS-libraries in ETP-package of ASSETS-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
logReturns Class initialization complete

Class initialization and load complete!


In [3]:
_from_date = '2022-01-01'
_to_date = '2022-01-31'
# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date >= '{_from_date}' and "+\
#         f"mcap_date <= '{_to_date}'"
_query = "select * from warehouse.mcap_past "+\
        f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
        f"and mcap_value > 1000000"
_kwargs = {
    "TABLENAME":'warehouse.mcap_past',
    "COLUMN":'mcap_date',
    "FROMDATETIME":_from_date,
    "TODATETIME":_to_date,
    "PARTITIONS":4
}

mcap_sdf = clsROR.read_n_clean_mcap(query=_query)
# mcap_sdf = clsROR.read_n_clean_mcap(**_kwargs)
print("Loaded %d rows" % mcap_sdf.count())

22/12/30 22:43:20 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
22/12/30 22:43:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/30 22:43:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Wait a moment, retrieving data ...


22/12/30 22:45:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


22/12/30 22:45:53 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


22/12/30 22:46:23 WARN DAGScheduler: Broadcasting large task binary with size 6.5 MiB


Loaded 31 rows


In [34]:
_col_subset = mcap_sdf.columns
_col_subset.remove('mcap_date')
_nan_counts_sdf = clsSpark.count_column_nulls(
    data=mcap_sdf,
    column_subset=_col_subset
)

In [35]:
# print(_nan_counts_sdf.count(),len(_nan_counts_sdf.columns))
_nan_counts_sdf.show()

22/12/30 23:48:17 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB


22/12/30 23:48:45 WARN DAGScheduler: Broadcasting large task binary with size 9.1 MiB


+------+----+----+-----+------+----+-------+-----+----+----+----------+----------+------------------+-----+--------+---------+------+------------+-----------------+----+------+-----------+----+-----------+--------+--------------+--------------+----+-----+-------------+---------+-----------+-------+------------+---------+----+------------+--------+-------+-------+---------+-----------+---------+--------+-----------+------+-----+--------+--------+------------+---------+-------------+-----------+------+--------------+-------------+-----------------+----------+------+-----+-----+----+------+----------+---------------------------+---------------+--------------+----+----+--------+----------+------+---------------+----+------+---------+--------+-----+------+-----------+----------+-------+---------+--------------+------------+--------+---------------------------+------+----+----------+------+-----+-------------+----+-----+-------+-------+-----+---+-----+----+-----+------+-------+--------------

In [45]:
from pyspark.sql.functions import col,isnan, when, count, countDistinct
# apply countDistinct on each column
col_counts = _nan_counts_sdf.select(sum(c).alias(c) for c in _nan_counts_sdf.columns)
# col_counts = _nan_counts_sdf.agg(*(countDistinct(col(c)).alias(c) for c in _nan_counts_sdf.columns)).collect()[0].asDict()

TypeError: Invalid argument, not a string or column: <generator object <genexpr> at 0x7f6494e7ae40> of type <class 'generator'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [3]:
''' DEPRECATED replaced with class function '''
__table__='warehouse.mcap_past'
__column__='mcap_date'
__l_bound__='2022-01-01'
__u_bound__='2022-01-31'
__query__ = "select * from warehouse.mcap_past "+\
            "where mcap_date >= '2022-01-01' and "+\
                    "mcap_date <= '2022-01-31'"

__KWARGS__ = {
    "PARTITIONS":10
}

mcap_sdf = clsSpark.read_data_from_table(select=__query__, **__KWARGS__)
# mcap_sdf = clsSpark.read_data_from_table(
#     db_table=__table__,
#     db_column=__column__,
#     lower_bound=__l_bound__,
#     upper_bound=__u_bound__,
#     **__KWARGS__)
print("Loaded %d rows" % mcap_sdf.count())

22/12/29 20:52:30 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
22/12/29 20:52:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/29 20:52:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Wait a moment, retrieving data ...


Loaded 1531943 rows


In [ ]:
# select the cols with count=1 in an array
cols_to_drop = [col for col in df.columns if col_counts[col] == 0 ]

In [48]:
# drop the selected column
_nan_counts_sdf.drop(*cols_to_drop).show()


Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "Spark Context Cleaner"
Exception in thread "refresh progress" java.lang.OutOfMemoryError: Java heap space
Exception in thread "netty-rpc-env-timeout" java.lang.OutOfMemoryError: Java heap space
Exception in thread "spark-listener-group-executorManagement" java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.Utils$.tryOrStopSparkContext(Utils.scala:1452)
	at org.apache.spark.scheduler.AsyncEventQueue$$anon$2.run(AsyncEventQueue.scala:96)
Exception in thread "SparkUI-91" java.lang.OutOfMemoryError: Java heap space


22/12/29 17:58:31 ERROR Utils: Uncaught exception in thread executor-heartbeater
java.lang.OutOfMemoryError: Java heap space
22/12/29 17:58:31 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 130125 ms exceeds timeout 120000 ms
22/12/29 17:58:31 WARN SparkContext: Killing executors is not supported by current scheduler.


Py4JJavaError: An error occurred while calling o81963.collectToPython.
: java.lang.OutOfMemoryError: Java heap space


In [12]:
# mcap_sdf.show(n=2,vertical=True)
# mcap_sdf.filter(mcap_sdf.mcap_value > 1000000).select("asset_name").distinct().show()

# _gt_mill_mcap = mcap_sdf.filter(mcap_sdf.mcap_value > 1000000)
pivot_mcap=mcap_sdf.groupBy("mcap_date").pivot("asset_name").sum("mcap_value")
print(pivot_mcap.count(),len(pivot_mcap.columns))

In [74]:
''' DEPRECATED moved to read-n-clean in logROR'''
_col_subset = pivot_mcap.columns
_col_subset.remove('mcap_date')
imp_mcap_sdf = clsSpark.impute_data(
    data=pivot_mcap,
    column_subset=_col_subset
)

22/12/30 21:32:53 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


22/12/30 21:33:17 WARN DAGScheduler: Broadcasting large task binary with size 6.5 MiB


In [76]:
''' DEPRECATED moved to sparkDBwls '''
from pyspark.sql.functions import col,isnan, when, count, countDistinct

_nulls_sdf = imp_mcap_sdf.drop('mcap_date')
# print(pivot_mcap._columns)
_nan_counts_sdf = _nulls_sdf.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in _nulls_sdf.columns])